In [1]:
# %matplotlib
# %matplotlib inline
import numpy
import matplotlib.pyplot as plt
import time

In [2]:
import sys
import os
sys.path.append('../..')
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:
import gillespy2
from gillespy2 import TauHybridSolver
from gillespy2 import NumPySSASolver
from gillespy2 import TauLeapingSolver


In [4]:
class SimpleHybridModel(gillespy2.Model):
     def __init__(self, parameter_values=None, init_v=1):
            #initialize Model
            gillespy2.Model.__init__(self, name="Simple_Hybrid_Model")

            
            #Species
            A = gillespy2.Species(name='A', initial_value=50)
            V = gillespy2.Species(name='V', initial_value=init_v)

            self.add_species([A, V])
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 20.0)
            rate2 = gillespy2.Parameter(name='rate2', expression= 10.0)
#             rate_rule1 = gillespy2.RateRule(V, "cos(t)")
            self.add_parameter([rate1, rate2])
#             self.add_rate_rule(rate_rule1)
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={}, products={A:1},
                   propensity_function="rate1 * V")
            
            r2 = gillespy2.Reaction(name="r2",reactants={A:1}, products={},
                    rate=rate2)
            
            self.add_reaction([r1,r2])
            self.timespan(numpy.linspace(0,100, 101))

In [5]:
v_range = range(1, 500)
def run_test(solver, v_range):
    run_data = []
    for n in v_range:
        model = SimpleHybridModel(init_v=n)
        time_start = time.perf_counter()
        model.run(solver=solver, show_labels=True)
        time_end = time.perf_counter()
        run_data.append(time_end-time_start)
    return run_data

In [ ]:
timing_data = {'basic':[], 'tau':[], 'hybrid_tau':[]}
%time timing_data['basic'] = run_test(NumPySSASolver(), v_range)
%time timing_data['tau'] = run_test(TauLeapingSolver(), v_range)
%time timing_data['hybrid_tau'] = run_test(TauHybridSolver(), v_range)
print(timing_data)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Time Comparison of Solvers")
plt.xlabel("initial Value of V (species count)")
plt.ylabel("Simulation Run Time (s)")
plt.plot(v_range, timing_data['basic'], label='basic')
plt.plot(v_range, timing_data['tau'], label='tau')
plt.plot(v_range, timing_data['hybrid_tau'], label='hybrid_tau')
plt.legend(loc='best')
plt.savefig("TimeComparisonOfSolvers.pdf")

In [ ]:
import pickle

In [ ]:
file_name = 'time_comparison_data'

file_object = open(file_name, 'rb')
data = pickle.load(file_object)
data.keys()
print('TAU RESULTS:\n', data['tau'])
print('HYBRID RESULTS:\n', data['hybrid_tau'])
print('BASIC RESULTS:\n', data['basic'])

In [ ]:
from wand.image import Image as WImage
img = WImage(filename='TimeComparisonOfSolvers.pdf')
img